In [1]:
import pandas as pd


In [2]:
df = pd.read_excel('Listed Companies at PSX - HK - 25AUG2022.xlsx')
df.head()

,Sector,Company Symbol,Company Name,Pays Regular Dividend?,Has Given Dividend?,Has Issued Right?,Has Given Bonus?,Shariah Status,Face Value,No of Ordinary Shares,No of Free Float Shares,Fiscal Year End,Registrar
0,Automobile Assembler,AGTL,Al-Ghazi Tractors Limited,NaN,Yes,No,No,Compliant,5.0,57964201,3608735.0,Dec,FAMCO Associates (Pvt.) Limited
1,Automobile Assembler,ATLH,Atlas Honda Limited,NaN,Yes,No,Yes,Compliant,10.0,124087935,12408794.0,Mar,Hameed Majeed Associates (Pvt.) Limited
2,Automobile Assembler,GHNL,Ghandara Nissan Limited,NaN,Yes,Yes,No,Compliant,10.0,57002500,19950875.0,Jun,CDC Share Registrar Services Limited
3,Automobile Assembler,GHNI,Ghandhara Industries Limited,NaN,Yes,No,Yes,Compliant,10.0,42608844,12782653.0,Jun,Hameed Majeed Associates (Pvt.) Limited
4,Automobile Assembler,HINO,HinoPak Motors Limited,NaN,Yes,Yes,No,Compliant,10.0,24801120,2546126.0,Mar,FAMCO Associates (Pvt.) Limited


In [3]:
df = df[df['Shariah Status'] == 'Compliant']

In [4]:
df['Final Name'] = df['Company Name'] + ' - ' + df['Company Symbol']

df['Final Name']

# (Colony) Thal Textile Mills Limited - COTT

0                       Al-Ghazi Tractors Limited - AGTL
1                             Atlas Honda Limited - ATLH
2                         Ghandara Nissan Limited - GHNL
3                    Ghandhara Industries Limited - GHNI
4                          HinoPak Motors Limited - HINO
                             ...                        
420        Pakistan International Bulk Terminal  - PIBTL
422    Pakistan National Shipping Corporation Limited...
423                      Punjab Oil Mills Limited - POML
424                        S.S. Oil Mills Limited - SSOM
425                   Bannu Woollen Mills Limited - BNWM
Name: Final Name, Length: 251, dtype: object

In [5]:
list_of_companies = list(df['Final Name'])

In [6]:
list_of_companies[0]

'Al-Ghazi Tractors Limited - AGTL'

In [15]:
from selenium import webdriver
from time import sleep
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
dynamic_path = os.getcwd()+'\data'

if not os.path.exists(dynamic_path):
    os.mkdir(dynamic_path)
    
for name in list_of_companies[:5]:
    try:
        if not os.path.exists(dynamic_path):
            os.mkdir(dynamic_path)
        
        download_path = dynamic_path  # Use double backslashes for Windows paths
        
        # Set Chrome options to specify the download directory
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_experimental_option('prefs', {
            'download.default_directory': download_path
        })
        # Start a Selenium webdriver session
        driver = webdriver.Chrome(options=chrome_options)

        # Navigate to a web page with the read-only input field
        driver.get('https://www.ksestocks.com/QuotationsData')  # Replace with the URL of your web page
        driver.implicitly_wait(3)
        value_to_select = 'csv'  # Replace with the value you want to select
        radio_button_excel = driver.find_element_by_xpath(f'//input[@type="radio" and @value="{value_to_select}"]')

        # Click on the radio button
        radio_button_excel.click()

        driver.implicitly_wait(3)
        value_to_select = 'byscrip'  # Replace with the value you want to select
        radio_button_script = driver.find_element_by_xpath(f'//input[@type="radio" and @value="{value_to_select}"]')

        # Click on the radio button
        radio_button_script.click()
        driver.implicitly_wait(3)


        dropdown_path = driver.find_element_by_class_name('optionselect') 
        dropdown = Select(dropdown_path)
        dropdown.select_by_visible_text(name)


        # Find the read-only input field by its HTML attribute (e.g., id, name, or XPath)
        read_only_input = driver.find_element_by_xpath('//*[@id="rfdate"]')  # Replace 'readonly_field' with the actual attribute value

        # Use JavaScript to set the value of the read-only input field
        new_value = '2013-01-01'  # Replace with the value you want to set
        driver.execute_script('arguments[0].value = arguments[1];', read_only_input, new_value)

        driver.implicitly_wait(3)
        value_to_click = 'Prepare Summary File'  # Replace with the value you want to click
        submit_button = driver.find_element_by_xpath(f'//input[@type="submit" and @value="{value_to_click}"]')

        # Click on the submit button
        submit_button.click()

        driver.implicitly_wait(3)
        download = driver.find_element_by_xpath('/html/body/div[2]/div/table/tbody/tr/td/div[2]/table/tbody/tr/td[1]/a/span')
        download.click()

        sleep(10)

        # Close the browser window
        driver.quit()

    except:
        continue

In [16]:

import os
import gzip
import shutil

source_folder = os.getcwd() + '\data'  # Replace with the path to your source folder

# Specify the target folder where you want to extract all the contents
target_folder = os.getcwd() + '\extracted data' # Replace with the path to your target folder

# Create the target folder if it doesn't exist
os.makedirs(target_folder, exist_ok=True)

# List all files in the source folder
files = os.listdir(source_folder)

# Iterate through the files in the source folder
for file in files:
    if file.endswith('.gz'):
        # Create the full path to the source .gz file
        source_file = os.path.join(source_folder, file)
        
        # Create the target file path by removing the .gz extension
        target_file = os.path.join(target_folder, os.path.splitext(file)[0])
        
        # Open the .gz file and extract its contents to the target folder
        with gzip.open(source_file, 'rb') as f_in:
            with open(target_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

print("Extraction completed.")


Extraction completed.


In [21]:
import os
import pandas as pd

# Specify the folder path containing the CSV files
folder_path = os.getcwd() + '\extracted data'

# Initialize an empty list to store DataFrames from each CSV file
dataframes = []

# Iterate through all files in the folder
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)  # Read each CSV file
            dataframes.append(df)  # Append each DataFrame to the list

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dataframes, ignore_index=True)




In [22]:
final_df

,Symbol,Date,Open,High,Low,Close,Volume
0,AABS,07-Jan-2013,103.50,103.50,103.50,103.50,500
1,AABS,14-Jan-2013,103.00,103.00,103.00,103.00,1000
2,AABS,16-Jan-2013,103.00,103.00,103.00,103.00,500
3,AABS,17-Jan-2013,108.00,108.00,105.75,105.75,1000
4,AABS,25-Feb-2013,96.70,96.70,96.70,96.70,1000
...,...,...,...,...,...,...,...
459148,ZTL,25-Oct-2023,9.60,9.60,9.60,9.60,500
459149,ZTL,26-Oct-2023,10.15,10.60,10.15,10.58,130000
459150,ZTL,27-Oct-2023,11.58,11.58,9.58,9.93,238000
459151,ZTL,30-Oct-2023,10.49,10.50,10.06,10.10,8500


In [23]:
final_df.to_csv('final data.csv', index=False)